# Alma Citation Item Report Generator

## Requirements
- Create a keys.env file within your runtime environment or source folder. Enter your bibs key (read only) as bibKey=bibkeyhere, and on a new line enter your courses key (read only) as courseKey=coursekeyhere.

- Generate an analytics report that contains the following columns: Citation Id, Course ID, Course Code, Course Instructor, Reading List Id, Current Course End Date, MMS Id. Leave column names default. Download as data > excel format. Name it citation ids.xlsx and upload it to the runtime environment.

- If you're using Google Colab to generate the report, you'll need to install both the asyncio-throttle and python-dotenv libraries. To do so, use the following pip commands: !pip install python-dotenv, !pip install asyncio-throttle.

### General Information

- Columns in final report: ['course code', 'title', 'author', 'instructor', 'edition','call number', 'isbn', 'barcode', 'permanent location', 'temp location', 'in temp location', 'temp call number', 'temp policy', 'due back date', 'course end date']

- Report includes the physical item information for all unique barcodes under a title heading, including ones not in a temp location at a reserve desk.

- Once the report is downloaded, select a cell in the area containing data and select "Format as Table". This will allow you to drill down to specific subsets of reserves or items.


In [ ]:
import aiohttp
import asyncio
from asyncio_throttle import Throttler
import pandas as pd
import os

In [ ]:
%load_ext dotenv
%dotenv keys.env

In [ ]:
df = pd.read_excel("citation ids.xlsx", dtype="str")
df.columns = [column.lower().replace(" ", "_") for column in df.columns]

In [ ]:
api_key = os.getenv("bibKey")
headers = {
    "Authorization": f"apikey {api_key}",
    "Accept": "application/json",
}
base = "https://api-na.hosted.exlibrisgroup.com/almaws/v1/bibs/"

In [ ]:
course_key = os.getenv("courseKey")

course_headers = {
    "Authorization": f"apikey {course_key}",
    "Accept": "application/json",
}

In [ ]:
# Function builds out the dictionary that becomes a row in the final dataframe. Contains physical item information.
def get_phys_item(
    i,
):  
    b = {
        "isbn": i["bib_data"]["isbn"],
        "title": i["bib_data"]["title"],
        "author": i["bib_data"]["author"],
        "barcode": i["item_data"]["barcode"],
        "call number": i["holding_data"]["call_number"],
        "permanent location": i["item_data"]["location"]["desc"],
        "edition": i["bib_data"]["complete_edition"],
    }

    try:
        b["due back date"] = i["holding_data"]["due_back_date"]
    except KeyError:
        b["due back date"] = "none"
    try:
        b["in temp location"] = i["holding_data"]["in_temp_location"]
    except KeyError:
        b["in temp location"] = "none"
    try:
        b["temp location"] = i["holding_data"]["temp_location"]["value"]
    except KeyError:
        b["temp location"] = "none"
    try:
        b["temp policy"] = i["holding_data"]["temp_policy"]["value"]
    except KeyError:
        b["temp policy"] = "none"
    try:
        b["temp call number"] = i["holding_data"]["temp_call_number"]
    except KeyError:
        b["temp call number"] = "none"

    return b

In [ ]:
# Creates the URL endpoints needed to make citation requests.
def build_url(df):  

    a = []

    baseURL = "https://api-na.hosted.exlibrisgroup.com/almaws/v1/courses/"

    for index, row in df.iterrows():
        course_id, citation_id, reading_list_id = (
            row["course_id"],
            row["citation_id"],
            row["reading_list_id"],
        )

        url = (
            baseURL
            + course_id
            + "/reading-lists/"
            + reading_list_id
            + "/citations/"
            + citation_id
        )

        a.append(url)

    return a

In [ ]:
# Async handler for making bibs (phys item) api calls. Works together with get_bibs()
async def get_bibs_list(
    df, headers
):  
    throttler = Throttler(rate_limit=25)
    async with aiohttp.ClientSession() as client:

        awaitables = [
            get_bibs(client=client, throttler=throttler, headers=headers, mms=i)
            for i in df["mms_id"]
        ]

        itemList = await asyncio.gather(*awaitables)

    return itemList

In [ ]:
# Works with get_bibs_list() to batch collection item information.
async def get_bibs(
    client, throttler, headers, mms
):  
    items = []
    async with throttler:
        try:
            resp = None
            url = base + mms + "/holdings/ALL/items"
            async with client.get(url, headers=headers) as session:
                if session.status != 200:
                    resp = await session.text()
                    session.raise_for_status()
                resp = await session.json()
                if (
                    resp["total_record_count"] == 0
                ):  # Checks to see how many items in the record. If none, adds the MMS ID of the record to the list so we can make citation calls on those MMS Ids
                    items.append(mms)
                else:
                    items.append(resp)
        except Exception as e:
            print(f"{e} for {mms}")

    return items

In [ ]:
# Used with get_ebook_list() to make batch citation calls to get ebook information.
async def get_ebook(
    client, throttler, headers, mms, df
):  
    ebooks = []
    async with throttler:
        try:
            resp = None
            courseInfo = df.loc[
                df["mms_id"] == mms
            ]  # Uses the MMS ID to pull the table row, then uses values from that row to apply course information (code, instructor, end date)
            url = str(courseInfo.url.values[0])
            async with client.get(url, headers=headers) as session:
                if session.status != 200:
                    resp = await session.text()
                    session.raise_for_status()
                resp = await session.json()
                b = {
                    "title": resp["metadata"][
                        "title"
                    ],  # Build out the citation information as a dictionary, appends it into the ebooks list.
                    "author": resp["metadata"]["author"],
                    "isbn": resp["metadata"]["isbn"],
                    "edition": resp["metadata"]["edition"],
                    "barcode": "none - ebook",
                    "call number": "none",
                    "permanent location": "none",
                    "due back date": "none",
                    "in temp location": "none",
                    "temp location": "none",
                    "temp policy": "none",
                    "temp call number": "none",
                    "course code": courseInfo["course_code"].values[0],
                    "course end date": courseInfo["current_course_end_date"].values[0],
                    "instructor": courseInfo["course_instructor"].values[0],
                }
                ebooks.append(b)
        except Exception as e:
            print(f"{e} for {mms}")

    return ebooks

In [ ]:
# Works with get_ebook() to make batch courses calls to retrieve citation information. Uses course_headers dict and course key.
async def get_ebook_list(
    elec_items, course_headers
):  
    throttler = Throttler(rate_limit=25)
    async with aiohttp.ClientSession() as client:

        awaitables = [
            get_ebook(
                client=client, throttler=throttler, headers=course_headers, mms=i, df=df
            )
            for i in elec_items
        ]

        itemList = await asyncio.gather(*awaitables)

    return itemList

In [ ]:
# Simple string formatter to remove special characters from the MMS ids.
def formatter(
    str,
):  
    str1 = str.replace("[", "")
    str2 = str1.replace("]", "")
    str3 = str2.replace("'", "")

    return str3

In [ ]:
# Build out each row for a physical item. Uses MMS ID to find the appropriate row 
# containing static course information (instructor, code, end date) and applies that to each item.
def get_physical_citations(
    p_list, df, cit_list
):  
    for i in p_list:
        mms = i[0]["item"][0]["bib_data"]["mms_id"]
        courseInfo = df.loc[df["mms_id"] == mms]

        for j in i[0]["item"]:
            a = get_phys_item(j)
            a["course code"] = courseInfo["course_code"].values[0]
            a["course end date"] = courseInfo["current_course_end_date"].values[0]
            a["instructor"] = courseInfo["course_instructor"].values[0]
            cit_list.append(a)

In [ ]:
# Separates out the MMS IDs for citation calls from the item records of physical item calls.
def build_lists(
    itemlist, phys_items, elec_items
):  
    for i in itemlist:
        try:
            if i[0]["total_record_count"] > 0:
                phys_items.append(i)
        except TypeError as error:
            elec_items.append(i)

In [ ]:
# Extracts out the returned citation records from the async ebooks function, then loads those into the final list.
def load_ebooks(
    ebooks, citationItems
):  
    for i in ebooks:
        for j in i:
            citationItems.append(j)

In [ ]:
# Build out our endpoint urls for making course calls (for citation information we use for ebooks)
df["url"] = build_url(
    df
)  

In [ ]:
# Make bibs calls to get physical item information.
itemlist = await get_bibs_list(
    df, headers
)  

In [ ]:
# Initialize empty lists we'll use.
phys_items, temp_elec_items, citationItems, elec_items = (
    [] for i in range(4)
)  

In [ ]:
# Separates out physical item information (phys_items), and MMS ids to pull citation information (temp_elec_items)
build_lists(
    itemlist, phys_items, temp_elec_items
)  

In [ ]:
# format the mms IDs
e = [formatter(str(i)) for i in temp_elec_items]  

In [ ]:
# Remove duplicate MMS id's
elec_items = [*set(e)]  

In [ ]:
# Make course api calls to retrieve citation information
ebooks = await get_ebook_list(
    elec_items, course_headers
)  

In [ ]:
# Extract out citation information and adds it to the final list.
load_ebooks(
    ebooks, citationItems
)  

In [ ]:
# Extract relevent physical item information from the records we received, then adds them to the citationItems list.
get_physical_citations(
    phys_items, df, citationItems
)  

In [ ]:
# Generate a dataframe from the collection of physical / electronic item dictionaries.
c = pd.DataFrame.from_records(
    citationItems
)  

In [ ]:
# Drop duplicate rows while keeping the first instance.
d = c.drop_duplicates(
    keep="first"
)  

In [ ]:
# Choose the columns we want in the final report, and export it as xlsx.
columns = [
    "course code",
    "title",
    "author",
    "instructor",
    "edition",
    "call number",
    "isbn",
    "barcode",  
    "permanent location",
    "temp location",
    "in temp location",
    "temp call number",
    "temp policy",
    "due back date",
    "course end date",
]
d = d[columns]
d = d.sort_values(by=["course code"])
d.to_excel("cts.xlsx", index=False)